* Домашние задание №1 Гриднева Константина Александровича
Дан файл с данными по клиентам и транзакциям: customer_and_transaction.xlsx.
Необходимо выполнить следующие пункты:
(2 балла) Продумать структуру базы данных и отрисовать в редакторе.
(2 балла) Нормализовать базу данных (1НФ — 3НФ), описав, к какой нормальной форме приводится таблица и почему таблица в этой нормальной форме изначально не находилась.
(3 балла) Создать все таблицы в DBeaver, указав первичные ключи к таблицам, правильные типы данных, могут ли поля быть пустыми или нет (использовать команду CREATE TABLE).
(3 балла) Загрузить данные в таблицы в соответствии с созданной структурой (использовать команду INSERT INTO или загрузить файлы, используя возможности инструмента DBeaver; в случае загрузки файлами приложить скрины, что данные действительно были залиты).

Задание 1 (Продумать структуру базы данных и отрисовать в редакторе.)

Table customers {
  id integer [primary key]
  first_name varchar
  last_name varchar
  gender varchar
  DOB timestamp
  job_id integer
  wealth_segment varchar
  deceased_indicator varchar
  owns_car varchar
  address varchar
  postcode integer
  state varchar
  country varchar
  property_valuation integer
}

Table transactions {
  id integer [primary key]
  customer_id integer
  transaction_date timestamp
  online_order bool
  order_status varchar
  id_product integer
}

Table products {
  id integer [primary key]
  product_id integer
  brand varchar
  product_line varchar
  product_class varchar
  product_size varchar
  standard_cost float
  list_price float
}

Table jobs {
  id integer [primary key]
  job_title varchar
  job_industry_category varchar
}



Ref: "products"."id" < "transactions"."id_product"

Ref: "customers"."id" < "transactions"."customer_id"

Ref: "customers"."job_id" < "jobs"."id"

Задание 2 (Нормализовать базу данных (1НФ — 3НФ), описав, к какой нормальной форме приводится таблица и почему таблица в этой нормальной форме изначально не находилась.)

1НФ - есть
2НФ - есть
3НФ - В таблице Покупатели, работа покупателей и продукты зависят не напрямую от id транзакции.
Вынесем их в отдельные таблицы

Задание 3 (Создать все таблицы в DBeaver, указав первичные ключи к таблицам, правильные типы данных, могут ли поля быть пустыми или нет (использовать команду CREATE TABLE).)

In [ ]:
import psycopg2 as pg
import pandas as pd
import numpy as np

In [ ]:

def create_tables():
    
    commands = (
    """ 
    CREATE TABLE jobs (
        "id" integer PRIMARY KEY,
        "job_title" varchar,
        "job_industry_category" varchar
    )
    """
    ,

    """ 
    CREATE TABLE products (
        "id" integer PRIMARY KEY,
        "product_id" integer NOT NULL,
        "brand" varchar,
        "product_line" varchar,
        "product_class" varchar,
        "product_size" varchar,
        "standard_cost" float,
        "list_price" float
    )
    """
    ,

    """
    CREATE TABLE customers (
        "id" integer PRIMARY KEY,
        "first_name" varchar,
        "last_name" varchar,
        "gender" varchar,
        "dob" varchar,
        "job_id" integer NOT NULL,
        "wealth_segment" varchar,
        "deceased_indicator" varchar,
        "owns_car" varchar,
        "address" varchar,
        "postcode" integer,
        "state" varchar,
        "country" varchar,
        "property_valuation" integer,
        FOREIGN KEY ("job_id") REFERENCES jobs("id")
    )
    """
    ,

    """
    CREATE TABLE transactions (
        "id" integer PRIMARY KEY,
        "customer_id" integer,
        "transaction_date" timestamp,
        "online_order" varchar,
        "order_status" varchar,
        "id_product" integer,
        FOREIGN KEY ("customer_id") REFERENCES customers("id"),
        FOREIGN KEY ("id_product") REFERENCES products("id")
    )
    """
    )
    try:
        with pg.connect(database="HW1",
                        user="HW_user",
                        password="123456",
                        host="localhost",
                        port="5433") as conn:
            with conn.cursor() as cur:
                    for command in commands:
                        cur.execute(command)
    except (pg.DatabaseError, Exception) as error:
        print(error)
        
create_tables()

In [ ]:
db_transaction = pd.read_excel("data/customer_and_transaction__2.xlsx", sheet_name = "transaction")
db_customer = pd.read_excel("data/customer_and_transaction__2.xlsx", sheet_name = "customer")

In [ ]:
db_jobs = db_customer.copy()

db_jobs = db_jobs.drop(columns=["customer_id", "first_name", "last_name", "gender", "DOB", "wealth_segment", "deceased_indicator", "owns_car", "address", "postcode", "state", "country", "property_valuation"])
db_jobs = db_jobs.drop_duplicates(ignore_index = True)
db_jobs['job_industry_category'].fillna("nan", inplace = True)
db_jobs['job_title'].fillna("nan", inplace = True)
postgres_insert_query = """ INSERT INTO jobs (id, job_title, job_industry_category)
                                       VALUES (%s,%s,%s)"""
try:
    with pg.connect(database="HW1",
                    user="HW_user",
                    password="123456",
                    host="localhost",
                    port="5433") as conn:
        with conn.cursor() as cur:
                for i in range(len(db_jobs)):
                    values = (i, db_jobs["job_title"][i], db_jobs["job_industry_category"][i])
                    cur.execute(postgres_insert_query, values)                   
except (pg.DatabaseError, Exception) as error:
    print(error)
    



In [ ]:
db_customer["job_id"] = 0
for i in range(len(db_customer)):
    for j in range(len(db_jobs)):
        if db_customer["job_industry_category"][i] == db_jobs["job_industry_category"][j]:
            db_customer["job_id"][i] = j
            continue


In [ ]:
for i in range(len(db_customer)):
    for j in range(len(db_jobs)):
        if db_customer["job_title"][i] == db_jobs["job_title"][j]:
            db_customer["job_id"][i] = j

In [ ]:
db_customer = db_customer.drop(columns=["job_title", "job_industry_category"])

In [ ]:
db_customer.info()

In [ ]:
postgres_insert_query = """ INSERT INTO customers (id, first_name, last_name, gender, DOB, job_id, wealth_segment, deceased_indicator, owns_car, address, postcode, state, country, property_valuation)
                                       VALUES (%s, %s, %s ,%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s , %s, %s)"""
try:
    with pg.connect(database="HW1",
                    user="HW_user",
                    password="123456",
                    host="localhost",
                    port="5433") as conn:
        with conn.cursor() as cur:
                for i in range(len(db_customer)):
                    values = ((str)(db_customer["customer_id"][i]), (str)(db_customer["first_name"][i]), (str)(db_customer["last_name"][i]), (str)(db_customer["gender"][i]), (db_customer["DOB"][i]), 
                              (str)(db_customer["job_id"][i]), (str)(db_customer["wealth_segment"][i]), (str)(db_customer["deceased_indicator"][i]),
                              (str)(db_customer["owns_car"][i]), (str)(db_customer["address"][i]), (str)(db_customer["postcode"][i]), (str)(db_customer["state"][i]), (str)(db_customer["country"][i]), (str)(db_customer["property_valuation"][i]))
                    cur.execute(postgres_insert_query, values)                   
except (pg.DatabaseError, Exception) as error:
    print(error)

In [ ]:
db_products = db_transaction.copy()

db_products = db_products.drop(columns=["transaction_id", "customer_id", "transaction_date", "online_order", "order_status"])
db_products = db_products.drop_duplicates(ignore_index = True)

db_products['brand'].fillna("nan", inplace = True)
db_products['product_line'].fillna("nan", inplace = True)
db_products['product_class'].fillna("nan", inplace = True)
db_products['product_size'].fillna("nan", inplace = True)
db_products['standard_cost'].fillna("nan", inplace = True)


In [ ]:
postgres_insert_query = """ INSERT INTO products (id, product_id, brand, product_line, product_class, product_size, standard_cost, list_price)
                                       VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
try:
    with pg.connect(database="HW1",
                    user="HW_user",
                    password="123456",
                    host="localhost",
                    port="5433") as conn:
        with conn.cursor() as cur:
                for i in range(len(db_products)):
                    values = (i, (str)(db_products["product_id"][i]), (str)(db_products["brand"][i]), (str)(db_products["product_line"][i]), (str)(db_products["product_class"][i]),
                              (str)(db_products["product_size"][i]), (str)(db_products["standard_cost"][i]), (str)(db_products["list_price"][i]))
                    cur.execute(postgres_insert_query, values)                   
except (pg.DatabaseError, Exception) as error:
    print(error)

In [ ]:
db_transaction["product_id"] = 0
db_transaction.info()

for i in range(len(db_transaction)):
    for j in range(len(db_products)):
        if db_transaction["product_id"][i] == db_transaction["product_id"][j]:
            if db_transaction["brand"][i] == db_transaction["brand"][j]:
                if db_transaction["product_line"][i] == db_transaction["product_line"][j]:
                    if db_transaction["product_class"][i] == db_transaction["product_class"][j]:
                        if db_transaction["product_size"][i] == db_transaction["product_size"][j]:
                            if db_transaction["standard_cost"][i] == db_transaction["standard_cost"][j]:
                                if db_transaction["list_price"][i] == db_transaction["list_price"][j]:
                                    db_transaction["product_id"][i] = j
                                    continue

In [ ]:
db_transaction = db_transaction.drop(columns=[ "brand", "product_line", "product_class", "product_size", "standard_cost", "list_price"])

In [ ]:
db_transaction.info()

In [ ]:
postgres_insert_query = """ INSERT INTO transactions (id, customer_id, transaction_date, online_order, order_status, id_product)
                                       VALUES (%s,%s,%s,%s,%s,%s)"""
try:
    with pg.connect(database="HW1",
                    user="HW_user",
                    password="123456",
                    host="localhost",
                    port="5433") as conn:
        with conn.cursor() as cur:
                for i in range(len(db_transaction)):
                    values = ((str)(db_transaction["transaction_id"][i]), (str)(db_transaction["customer_id"][i]), (db_transaction["transaction_date"][i]), (str)(db_transaction["online_order"][i]),
                              (str)(db_transaction["order_status"][i]), (str)(db_transaction["product_id"][i]))
                    cur.execute(postgres_insert_query, values)                   
except (pg.DatabaseError, Exception) as error:
    print(error)

In [ ]:
postgres_insert_query = """ INSERT INTO customers (id, first_name, last_name, gender, DOB, job_id, wealth_segment, deceased_indicator, owns_car, address, postcode, state, country, property_valuation)
                                       VALUES (%s, %s, %s ,%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s , %s, %s)"""
try:
    with pg.connect(database="HW1",
                    user="HW_user",
                    password="123456",
                    host="localhost",
                    port="5433") as conn:
        with conn.cursor() as cur:
                values = ((str)(5034), "nan", "nan","nan", "nan", 
                            0, "nan", "nan",
                            "nan", "nan", 0, "nan", "nan", 0)
                cur.execute(postgres_insert_query, values)                   
except (pg.DatabaseError, Exception) as error:
    print(error)

In [ ]:
postgres_insert_query = """ INSERT INTO transactions (id, customer_id, transaction_date, online_order, order_status, id_product)
                                       VALUES (%s,%s,%s,%s,%s,%s)"""
try:
    with pg.connect(database="HW1",
                    user="HW_user",
                    password="123456",
                    host="localhost",
                    port="5433") as conn:
        with conn.cursor() as cur:
                for i in range(len(db_transaction)):
                    values = ((str)(db_transaction["transaction_id"][i]), (str)(db_transaction["customer_id"][i]), (db_transaction["transaction_date"][i]), (str)(db_transaction["online_order"][i]),
                              (str)(db_transaction["order_status"][i]), (str)(db_transaction["product_id"][i]))
                    cur.execute(postgres_insert_query, values)                   
except (pg.DatabaseError, Exception) as error:
    print(error)